# AlexNet

In [ ]:
from __future__ import adsolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
import tf_slim as slim

trunc_normal = lambda stddev: tf.truncated_normal_initializer(0.0, stddev)

def alexnet_v2_arg_scope(weight_decay=0.0005):
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                                          activation_fn=tf.nn.relu,
                                          biases_initializer=tf.constant_initializer(0.1),
                                          weights_regularizer=slim.l2_regularizer(weight_decay)):
         with slim.arg_scope([slim.conv2d], padding='SAME'):
                with slim.arg_scope([slim.max_pool2d], padding='VALID') as arg_sc:
                    return arg_sc

def alexnet_v2(inputs,
                            num_classes=1000,
                            is_training=True,
                            dropput_keep_prob=0.5,
                            spatial_squeeze=True,
                            scope='alexnet_v2',
                            global_pool=False):

    with tf.variable_scope(scope, 'alexnet_v2', [inputs]) as sc:
        end_points_collection = sc.original_name_scope + '_end_points'
        # Collect outputs for conv2d, fully_connected and max_pool2d.
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                                            outputs_collections=[end_points_collection]):
            net = slim.conv2d(inputs, 64, [11, 11], 4, padding='VALID',
                                           scope='conv1')
            net = slim.max_pool2d(net, [3, 3], 2, scope='pool1')
            net = slim.conv2d(net, 192, [5, 5], scope='conv2')
            net = slim.max_pool2d(net, [3, 3], 2, scope='pool2')
            net = slim.conv2d(net, 384, [3, 3], scope='conv3')
            net = slim.conv2d(net, 384, [3, 3], scope='conv4')
            net = slim.conv2d(net, 256, [3, 3], scope='conv5')
            net = slim.max_pool2d(net, [3, 3], 2, scope='pool5')
            
            # Use conv2d instead of fully_connected layers.
            with slim.arg_scope(
                    [slim.conv2d],
                     weights_initializer=trunc_normal(0.005),
                     biases_initializer=tf.constant_initializer(0.1)):
                net = slim.conv2d(net, 4096, [5, 5], padding='VALID',
                                                 scope='fc6')
                net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                                                  scope='dropput6')
                net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
                # Convert endpoints_collection into a end_point dict.
                end_points = slim.utils.convert_collection_to_dict(end_points_collection)
                if global_pool:
                    net = tf.reduce_mean(input_tensor=net, axis=[1, 2], keepdims=True, name='global_pool')
                    end_points['global_pool'] = net
                if num_classes:
                    net = slim.dropout(net, dropout_keep_prob, is_training=is_training, scope='dropout7')
                    net = slim.conv2d(
                           net, 
                           num_classes, [1,1],
                           activation_fn=None,
                           normalizer_fn=None,
                           biases_initializer=tf.zeros_initializer(),
                           scope='fc8')
                    if spatial_squeeze:
                        net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
                    end_points[sc.name + '/fc8'] = net
            return net, end_points
alexnet_v2.default_image_size = 224